EXPERIMENTS CO-1 (1-5)

In [2]:
data = [
    ["Red","Small","High","Soft","Yes"],
    ["Green","Small","Low","Hard","No"],
    ["Yellow","Large","High","Soft","Yes"],
    ["Red","Large","Medium","Hard","No"],
    ["Yellow","Small","Medium","Soft","Yes"],
    ["Green","Large","Low","Hard","No"]
]

n = len(data[0]) - 1
S = ["Ø"] * n

for row in data:
    if row[-1] == "Yes":
        for i in range(n):
            if S[i] == "Ø": S[i] = row[i]
            elif S[i] != row[i]: S[i] = "?"

print("Final Hypothesis (Find-S):", S)


Final Hypothesis (Find-S): ['?', '?', '?', 'Soft']


In [3]:
data = [
    ["Red","Small","High","Soft","Yes"],
    ["Green","Small","Low","Hard","No"],
    ["Yellow","Large","High","Soft","Yes"],
    ["Red","Large","Medium","Hard","No"],
    ["Yellow","Small","Medium","Soft","Yes"],
    ["Green","Large","Low","Hard","No"]
]

n = len(data[0]) - 1
S = ["Ø"] * n
G = [["?"] * n]

for row in data:
    x, y = row[:-1], row[-1]

    if y == "Yes":
        G = [g for g in G if all(g[i]=="?" or g[i]==x[i] for i in range(n))]
        for i in range(n):
            if S[i]=="Ø": S[i]=x[i]
            elif S[i]!=x[i]: S[i]="?"
    else:
        newG = []
        for g in G:
            if all(g[i]=="?" or g[i]==x[i] for i in range(n)):
                for i in range(n):
                    if g[i]=="?" and S[i]!="?" and S[i]!=x[i]:
                        h = g.copy(); h[i]=S[i]; newG.append(h)
            else:
                newG.append(g)
        G = newG

print("S Boundary:", S)
print("G Boundary:", G)


S Boundary: ['?', '?', '?', 'Soft']
G Boundary: [['?', '?', '?', 'Soft']]


In [4]:
# Inductive Bias Study – Find-S vs Majority

data = [
    ["Sunny","Hot","High","Weak","No"],
    ["Sunny","Cool","Normal","Weak","Yes"],
    ["Rain","Mild","High","Weak","Yes"],
    ["Overcast","Cool","Normal","Strong","Yes"],
    ["Rain","Mild","Normal","Weak","Yes"],
    ["Sunny","Hot","High","Strong","No"]
]

n = len(data[0]) - 1

# ----- Find-S -----
S = ["Ø"] * n
for row in data:
    if row[-1] == "Yes":
        for i in range(n):
            if S[i] == "Ø": S[i] = row[i]
            elif S[i] != row[i]: S[i] = "?"

# ----- Majority Classifier -----
labels = [row[-1] for row in data]
majority = max(set(labels), key=labels.count)

print("Find-S Hypothesis:", S)
print("Majority Class:", majority)


Find-S Hypothesis: ['?', '?', '?', '?']
Majority Class: Yes


In [5]:
# ID3 Decision Tree Algorithm

import math
from collections import Counter

data = [
    ["Sunny","Hot","High","Weak","No"],
    ["Sunny","Hot","High","Strong","No"],
    ["Overcast","Hot","High","Weak","Yes"],
    ["Rain","Mild","High","Weak","Yes"],
    ["Rain","Cool","Normal","Weak","Yes"],
    ["Rain","Cool","Normal","Strong","No"],
    ["Overcast","Cool","Normal","Strong","Yes"]
]

labels = ["Outlook","Temp","Humidity","Wind"]

def entropy(rows):
    total = len(rows)
    cnt = Counter(r[-1] for r in rows)
    return -sum((v/total)*math.log2(v/total) for v in cnt.values())

def gain(rows, col):
    base = entropy(rows)
    vals = set(r[col] for r in rows)
    weighted = sum((len([r for r in rows if r[col]==v]) / len(rows)) *
                   entropy([r for r in rows if r[col]==v]) for v in vals)
    return base - weighted

def id3(rows, cols):
    classes = [r[-1] for r in rows]
    if len(set(classes)) == 1:
        return classes[0]
    if not cols:
        return Counter(classes).most_common(1)[0][0]

    best = max(cols, key=lambda c: gain(rows, c))
    tree = {labels[best]: {}}

    for v in set(r[best] for r in rows):
        subset = [r for r in rows if r[best] == v]
        tree[labels[best]][v] = id3(subset, [c for c in cols if c != best])

    return tree

print("Decision Tree:", id3(data, list(range(len(labels)))))


Decision Tree: {'Outlook': {'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}, 'Sunny': 'No', 'Overcast': 'Yes'}}


In [6]:
data = [
    ["Red","Small","High","Soft","Yes"],
    ["Green","Small","Low","Hard","No"],
    ["Yellow","Large","High","Soft","Yes"],
    ["Red","Large","Medium","Hard","No"],
    ["Yellow","Small","Medium","Soft","Yes"],
    ["Green","Large","Low","Hard","No"]
]

n = len(data[0]) - 1
attrs = [set(r[i] for r in data) for i in range(n)]

def acc(h):
    return sum(
        ("Yes" if all(h[i]=="?" or h[i]==r[i] for i in range(n)) else "No") == r[-1]
        for r in data
    ) / len(data)

def neigh(h):
    ns=[]
    for i in range(n):
        for v in attrs[i]:
            if h[i]!=v:
                nh=h.copy(); nh[i]=v; ns.append(nh)
        nh=h.copy(); nh[i]="?"; ns.append(nh)
    return ns

current=["?"]*n
best=acc(current)

while True:
    scored=[(acc(h),h) for h in neigh(current)]
    score, cand = max(scored)
    if score > best:
        current, best = cand, score
    else:
        break

print("Best Hypothesis:", current)
print("Accuracy:", best)


Best Hypothesis: ['?', '?', '?', 'Soft']
Accuracy: 1.0
